In [47]:
import sys
#sys.path.append("tools/")
from time import time

In [48]:

original = 'word_data.pkl'
destination = "word_data_unix.pkl"

content = ''
outsize = 0
with open(original, 'rb') as infile:
    content = infile.read()
with open(destination, 'wb') as output:
    for line in content.splitlines():
        outsize += len(line) + 1
        output.write(line + str.encode('\n'))

print("Done. Saved %s bytes." % (len(content)-outsize))

Done. Saved 35156 bytes.


In [51]:
# Email preprocess file - email_preprocess.py
import pickle
#import cPickle
import numpy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif



def preprocess(words_file = "word_data_unix.pkl", authors_file="email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    #authors_file_handler = open(authors_file, "r")
    #authors = pickle.load(authors_file_handler)
    #authors_file_handler.close()
    
    with open(authors_file, 'rb') as f:
        authors = pickle.load(f)
    #print(authors[0])

    #words_file_handler = open(words_file, "r")
    #word_data = cPickle.load(words_file_handler)
    #words_file_handler.close()
    with open(words_file, 'rb') as w:
        word_data = pickle.load(w)
    #print(word_data[0])


    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print("no. of Chris training emails:", sum(labels_train))
    print("no. of Sara training emails:", len(labels_train)-sum(labels_train))
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test

In [52]:
### features_train and features_test are the features for the training
### and testing datasets, respectively
### labels_train and labels_test are the corresponding item labels
features_train, features_test, labels_train, labels_test = preprocess()



#########################################################
### your code goes here ###
from sklearn.naive_bayes import GaussianNB
classifier=GaussianNB()
t0 = time()
classifier.fit(features_train,labels_train)
print("training time:", round(time()-t0, 3), "s")
t0 = time()
pred=classifier.predict(features_test)
print("prediction time:", round(time()-t0, 3), "s")
from sklearn import metrics
accuracy = metrics.accuracy_score(labels_test, pred)
print(classifier.score(features_test, labels_test))
print(accuracy)

#########################################################

no. of Chris training emails: 7936
no. of Sara training emails: 7884
training time: 0.963 s
prediction time: 0.151 s
0.9732650739476678
0.9732650739476678
